In [38]:
import pandas as pd
import numpy as np
import pickle
import torch
from sklearn.neighbors import NearestNeighbors

In [39]:
def split_features_and_labels(dataframe: pd.DataFrame) -> pd.DataFrame:
    features = dataframe.iloc[:, :-11]  # All columns except the last eleven
    labels = dataframe.iloc[:, -11:]    # The last eleven columns
    return features, labels

In [40]:
# Load embeddings data
embeddings_path = 'data/image_embeddings.pt'
# load with torch.load
embeddings_df = torch.load(embeddings_path)


/tmp/ipykernel_17884/1554930739.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings_df = torch.load(embeddings_path)


In [41]:
embeddings_path_test = 'data/image_embeddings_test.pt'
embeddins_test = torch.load(embeddings_path_test)

/tmp/ipykernel_17884/1282979331.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddins_test = torch.load(embeddings_path_test)


In [42]:
# # Get the first ID and vector
# first_id = next(iter(embeddings_df))
# first_vector = embeddings_df[first_id]

# # Print the first ID and vector
# print(f"ID: {first_id}")
# print(f"Vector: {first_vector}")

In [43]:
import torch
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

# Load data
test_data = pd.read_csv('data/test_data.csv')
embeddings_df = torch.load('data/image_embeddings.pt')
embeddings_test = torch.load('data/image_embeddings_test.pt')

# Prepare all embeddings at once
print("Preparing embeddings...")
image_ids = list(embeddings_df.keys())
embedding_vectors = np.array([embeddings_df[img].numpy() for img in image_ids])

# Prepare test embeddings in batch
test_image_paths = test_data['des_filename'].tolist()  # Adjust column name if different
valid_test_images = [img for img in test_image_paths if img in embeddings_test]
test_embedding_vectors = np.array([embeddings_test[img].numpy() for img in valid_test_images])

# Initialize and fit KNN
print("Fitting KNN...")
knn = NearestNeighbors(n_neighbors=1, algorithm='auto', n_jobs=-1)  # Use all CPU cores
knn.fit(embedding_vectors)

# Find all nearest neighbors at once
print("Finding nearest neighbors...")
distances, indices = knn.kneighbors(test_embedding_vectors)

# Create results DataFrame efficiently
print("Creating results DataFrame...")
results = []
for i, test_img in enumerate(valid_test_images):
    neighbors = indices[i]
    neighbor_distances = distances[i]
    for rank, (neighbor_idx, dist) in enumerate(zip(neighbors, neighbor_distances), 1):
        results.append({
            'test_image': test_img,
            'neighbor_rank': rank,
            'neighbor_id': image_ids[neighbor_idx],
            'distance': dist
        })

results_df = pd.DataFrame(results)

# Save results
results_df.to_csv('nearest_neighbors_results.csv', index=False)

print("\nResults Summary:")
print(f"Total test images processed: {len(valid_test_images)}")
print(f"Total neighbors found: {len(results_df)}")

/tmp/ipykernel_17884/346149815.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings_df = torch.load('data/image_embeddings.pt')
/tmp/ipykernel_17884/346149815.py:1

Preparing embeddings...
Fitting KNN...
Finding nearest neighbors...
Creating results DataFrame...

Results Summary:
Total test images processed: 71819
Total neighbors found: 71819


In [19]:
knn_results = pd.read_csv('nearest_neighbors_results.csv')
products = pd.read_csv('data/product_with_attributes.csv')

sample_submission = pd.read_csv('../archive/sample_submission.csv')
submission = sample_submission.copy()

# Merge knn_results with products on des_filename to get neighbor values
neighbor_values = knn_results.merge(products, left_on='neighbor_id', right_on='des_filename', how='left')
neighbor_values['product_id'] = neighbor_values['test_image'].str.split('_').str[:2].str.join('_')


In [ ]:
unique_neighbor_values = neighbor_values.drop_duplicates(subset='product_id').set_index('product_id')


In [23]:
unique_neighbor_values

,test_image,neighbor_rank,neighbor_id,distance,cod_modelo_color,des_sex,des_age,des_line,des_fabric,des_product_category,...,neck_lapel_type,woven_structure,knit_structure,heel_shape_type,length_type,sleeve_length_type,toecap_type,waist_type,closure_placement,cane_height_type
product_id,,,,,,,,,,,,,,,,,,,,,
88_49711373,88_49711373_67080432-99_.jpg,1,85_1190636_37094384-99_.jpg,5.492573,85_1190636,Female,Adult,WOMAN,ACCESSORIES,"Accesories, Swim and Intimate",...,INVALID,INVALID,INVALID,Plano,INVALID,INVALID,Redonda,INVALID,INVALID,Alta
88_49718802,88_49718802_67030656-99_.jpg,1,86_1205150_47080568-99_.jpg,5.176957,86_1205150,Female,Adult,WOMAN,ACCESSORIES,"Accesories, Swim and Intimate",...,INVALID,INVALID,INVALID,Bloque,INVALID,INVALID,Con punta,INVALID,INVALID,Media
88_49709572,88_49709572_67030418-01_B.jpg,1,82_1115175_87006338-02_B.jpg,4.726316,82_1115175,Female,Kids,KIDS,CIRCULAR,Tops,...,Redondo,Ligero,INVALID,INVALID,Standard,Corta,INVALID,INVALID,INVALID,INVALID
88_49722701,88_49722701_67066002-02_.jpg,1,85_1176791_37000164-68_.jpg,5.882247,85_1176791,Female,Baby,KIDS,JEANS,Bottoms,...,INVALID,Medio,INVALID,INVALID,Corto,INVALID,INVALID,Ajustable/Goma,INVALID,INVALID
88_49724926,88_49724926_67056330-02_B.jpg,1,86_1228579_47087840-01_B.jpg,3.412920,86_1228579,Male,Newborn,KIDS,WOVEN,Tops,...,Redondo,Ligero,INVALID,INVALID,Standard,Corta,INVALID,INVALID,INVALID,INVALID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88_49735572,88_49735572_67076755-81_.jpg,1,83_1120896_17010101-85_.jpg,5.972844,83_1120896,Female,Adult,WOMAN,CIRCULAR,Tops,...,Pico,Ligero,INVALID,INVALID,Standard,Corta,INVALID,INVALID,INVALID,INVALID
88_49713624,88_49713624_67092528-70_.jpg,1,85_1181950_37001111-87_.jpg,5.985125,85_1181950,Female,Adult,WOMAN,WOVEN,"Dresses, jumpsuits and Complete set",...,Pico,Ligero,INVALID,INVALID,Corto,Tirante Fino,INVALID,INVALID,Sin cierre,INVALID
88_49718191,88_49718191_67084448-03_.jpg,1,86_1218756_47084412-06_.jpg,5.654564,86_1218756,Male,Adult,MAN,WOVEN,Outerwear,...,Chimenea,Ligero,INVALID,INVALID,Corto,Larga,INVALID,INVALID,Cierre Delantero,INVALID


In [35]:
for index, row in tqdm(submission.iterrows()):
    splitted = row['test_id'].split('_')
    product_id = '_'.join(splitted[:2])
    attribute_name = '_'.join(splitted[2:])
    # print(product_id, attribute_name)
    row['des_value'] = unique_neighbor_values.loc[product_id][attribute_name]


71819it [00:24, 2905.41it/s]


In [ ]:
neighbor_values[neighbor_values['product_id'] == '88_49726492']

In [37]:
submission.to_csv('submission_knn.csv', index=False)

In [36]:
submission

,test_id,des_value
0,88_49726492_silhouette_type,Recto
1,88_49717057_silhouette_type,Oversize
2,88_49727807_silhouette_type,Slim
3,88_49718986_silhouette_type,Regular
4,88_49716369_silhouette_type,Recto
...,...,...
71814,88_49727621_toecap_type,INVALID
71815,88_49727736_toecap_type,INVALID
71816,88_49734138_toecap_type,INVALID
71817,88_49717732_toecap_type,INVALID
